In [ ]:
import pandas as pd
import numpy as np

from itertools import combinations
from math import radians, sin, cos, sqrt, atan2

In [ ]:
# Importacao da base de dados contendo a latitude e a longitude de cada municipio

tabela = pd.read_csv('municipios.csv')

In [ ]:
tabela.head()
codigo_ibge	latitude	longitude	capital	codigo_uf	siafi_id	ddd
0	5200050	-16.75730	-49.4412	0	52	1050	62
1	3100104	-18.48310	-47.3916	0	31	4001	34
2	5200100	-16.19700	-48.7057	0	52	9201	62
3	3100203	-19.15510	-45.4444	0	31	4003	37
4	1500107	-1.72183	-48.8788	0	15	401	91


In [ ]:
# Funcao para converter coordenadas de latitude e longitude para cartesianas

def lat_long_to_cartesiana(lat, lon):
    R = 6371 # Raio medio da Terra em quilometros
    x = R * np.cos(np.radians(lat)) * np.cos(np.radians(lon))
    y = R * np.cos(np.radians(lat)) * np.sin(np.radians(lon))
    z = R * np.sin(np.radians(lat))

    return x,y,z

In [ ]:
df = pd.DataFrame(tabela)

# Aplicar a função de conversão para obter as coordenadas cartesianas
df['x'], df['y'], df['z'] = lat_long_to_cartesiana(df['latitude'], df['longitude'])

# Exibindo o DataFrame com as coordenadas cartesianas
print(df)
      codigo_ibge  latitude  longitude  capital  codigo_uf  siafi_id  ddd  \
0         5200050 -16.75730   -49.4412        0         52      1050   62   
1         3100104 -18.48310   -47.3916        0         31      4001   34   
2         5200100 -16.19700   -48.7057        0         52      9201   62   
3         3100203 -19.15510   -45.4444        0         31      4003   37   
4         1500107  -1.72183   -48.8788        0         15       401   91   
...           ...       ...        ...      ...        ...       ...  ...   
5565      2933604 -10.82300   -42.7245        0         29      3971   74   
5566      2517407  -8.07901   -37.1057        0         25       542   83   
5567      3557154 -21.05060   -50.0552        0         35      2973   18   
5568      2114007  -3.27014   -45.6553        0         21      1287   98   
5569      4219853 -27.45210   -51.5520        0         42       950   49   

                x            y            z  
0     3966.685915 -4634.752587 -1836.875692  
1     4090.584393 -4447.168365 -2019.765794  
2     4037.514851 -4596.728913 -1777.131995  
3     4222.419033 -4288.432592 -2090.493821  
4     4188.021993 -4797.235388  -191.429944  
...           ...          ...          ...  
5565  4597.039687 -4245.667120 -1196.318442  
5566  5030.596759 -3805.397224  -895.370992  
5567  3817.510930 -4558.444070 -2288.414072  
5568  4445.908552 -4548.786650  -363.425591  
5569  3515.433833 -4427.748486 -2937.074958  

[5570 rows x 10 columns]

In [ ]:
# Função para calcular a distância Haversine entre dois pares de latitude e longitude
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Earth radius in kilometers
    return distance

In [ ]:
# Lista para armazenar as distâncias entre as cidades
distances = []

# Calcular a distância entre cada par de cidades
for codigo_ibge_1, codigo_ibge_2 in combinations(df.index, 2):
    lat1, lon1 = df.loc[codigo_ibge_1, ['latitude', 'longitude']]
    lat2, lon2 = df.loc[codigo_ibge_2, ['latitude', 'longitude']]
    distance = haversine(lat1, lon1, lat2, lon2)
    distances.append((codigo_ibge_1, codigo_ibge_2, distance))

# Criar DataFrame com as distâncias calculadas
distances_df_2 = pd.DataFrame(distances, columns=['City1', 'City2', 'Distance (km)'])
print(distances_df_2)

In [ ]:
distances_df_2.to_csv('base_distancias_geral.csv')

In [ ]:
distances_df_2
City1	City2	Distance (km)
0	0	1	289.830897
1	0	2	100.160142
2	0	3	499.787149
3	0	4	1672.999956
4	0	5	1539.696725
...	...	...	...
15509660	5566	5568	1086.421409
15509661	5566	5569	2636.324098
15509662	5567	5568	2033.605580
15509663	5567	5569	727.783339
15509664	5568	5569	2760.915335
15509665 rows × 3 columns

PARTE 2: CALCULO CONSIDERANDO APENAS AS CAPITAIS

In [ ]:
# Capitais de interesse

codigo_ibge_estados = [
    "1100205", "1302603", "1200401", "5002704", "1600303", "5300108",
    "1400100", "5103403", "1721000", "3550308", "2211001", "3304557",
    "1501402", "5208707", "2927408", "4205407", "2111300", "2704302",
    "4314902", "4106902", "3106200", "2304400", "2611606", "2507507",
    "2800308", "2408102", "3205309"
]

In [ ]:
# Para esta base especifica foi necessario ajustar o COD IBGE para 6 digitos
distances_df['City1'] = distances_df['City1'].astype(str).str[:6]
distances_df['City2'] = distances_df['City2'].astype(str).str[:6]

In [ ]:
# Converter a coluna 'City1' para tipo str
distances_df['City1'] = distances_df['City1'].astype(str)

# Filtrar as cidades na coluna 'City1' usando a lista de códigos IBGE de estados específicos
filtered_distances_df = distances_df[distances_df['City1'].isin(codigo_ibge_estados)]

# Exibir o DataFrame filtrado
print(filtered_distances_df)
Empty DataFrame
Columns: [City1, City2, Distance (km)]
Index: []

In [ ]:
# Encontrar os índices das linhas com as menores distâncias para cada 'City2'
min_distance_indices = filtered_distances_df.groupby('City2')['Distance (km)'].idxmin()

# Filtrar o DataFrame original usando os índices encontrados
closest_distances_df = filtered_distances_df.loc[min_distance_indices]

# Exibir o DataFrame com os menores valores de distância para cada 'City2'
print(closest_distances_df)
Empty DataFrame
Columns: [City1, City2, Distance (km)]
Index: []